In [11]:
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from warnings import simplefilter
from sktime.datasets import load_airline
from sktime.forecasting.arima import ARIMA, AutoARIMA
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.compose import (
    EnsembleForecaster,
    MultiplexForecaster,
    TransformedTargetForecaster,
    make_reduction,
)
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.model_evaluation import evaluate
from sktime.forecasting.model_selection import (
    ExpandingWindowSplitter,
    ForecastingGridSearchCV,
    SlidingWindowSplitter,
    temporal_train_test_split,
)
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.theta import ThetaForecaster
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.performance_metrics.forecasting import (
    MeanAbsolutePercentageError,
    mean_absolute_percentage_error,
)
from sktime.transformations.series.detrend import Deseasonalizer, Detrender
from sktime.utils.plotting import plot_series

simplefilter("ignore", FutureWarning)
%matplotlib inline

if __name__ == "__main__":
    target_feature = "processingLatencyMean"
    f = open("stats-Conjunction221998ed94--1593549630.csv")
    df = pd.read_csv(f, parse_dates=["timestamp"], sep=";", header=0)
    f.close()
    df.set_index("timestamp", inplace=True)
    df.index.freq = "S"
    print(df.index)

    #df = df._get_numeric_data()
    ts = df[target_feature][:]
    plot_series(ts)
    print(ts)
    y_train, y_test = temporal_train_test_split(ts, test_size=3)
    plot_series(y_train, y_test, labels=["y_train", "y_test"])
    print(y_train.shape[0], y_test.shape[0])


    fh = np.arange(len(y_test)) + 1

ValueError: Inferred frequency None from passed values does not conform to passed frequency S

In [ ]:
    from sklearn.neighbors import KNeighborsRegressor

    regressor = KNeighborsRegressor(n_neighbors=1)
    forecaster = make_reduction(regressor, window_length=5, strategy="recursive")
    forecaster.fit(y_train)
    y_pred = forecaster.predict(fh)
    plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"])
    mean_absolute_percentage_error(y_pred, y_test)

In [ ]:
    forecaster = AutoARIMA()
    forecaster.fit(y_train)
    y_pred = forecaster.predict(fh)
    plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"])
    mean_absolute_percentage_error(y_pred, y_test)




In [ ]:
    from sktime.forecasting.bats import BATS

    forecaster = BATS(sp=None, use_trend=True, use_box_cox=False)
    forecaster.fit(y_train)
    y_pred = forecaster.predict(fh)
    plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"])
    mean_absolute_percentage_error(y_pred, y_test)


In [ ]:
    # Convert index to pd.DatetimeIndex
    z = ts.copy()
    #z = z.to_timestamp(freq="S")
    z_train, z_test = temporal_train_test_split(z, test_size=3)
    from sktime.forecasting.fbprophet import Prophet

    forecaster = Prophet(
        #seasonality_mode="multiplicative",
        #n_changepoints=int(len(y_train) / 12),
        #add_country_holidays={"country_name": "Germany"},
        #yearly_seasonality=True,
        weekly_seasonality=False,
        daily_seasonality=False,
    )
    forecaster.fit(z_train)
    y_pred = forecaster.predict(fh.to_relative(cutoff=y_train.index[-1]))

    y_pred.index = y_test.index
    plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"])
    mean_absolute_percentage_error(y_pred, y_test)